[`pidgin.weave`](weave.md.ipynb) allows code to be execute with the display function.

In [1]:
    if __name__ == '__main__': 
        %reload_ext pidgin

In [2]:
    import ast, jinja2, sys, nbconvert, nbformat, contextlib, collections, IPython, mistune, doctest, yaml, re, traitlets, textwrap, types, typing as t, importlib, importnb,  jinja2, builtins, jsonschema
    from pidgin import tangle


In [3]:
    
    class HtmlGrammar: 
        block_html = re.compile(
            r'^ *(?:%s|%s|%s) *(?:\n{2,}|\s*$)' % (
                r'<!--[\s\S]*?-->|<!DOCTYPE [\s\S]*?>|<\?[\s\S]*?\?>',
                r'<(%s)((?:%s)*?)>([\s\S]*?)<\/\1>' % (mistune._block_tag, mistune._valid_attr),
                r'<%s(?:%s)*?\s*\/?>' % (mistune._block_tag, mistune._valid_attr)))       

    class InlineHtmlGrammar:
        """Add svg constructs to inline_html regex."""
        inline_html = re.compile(
                r'^(?:%s|%s|%s)' % (
                    r'<!--[\s\S]*?-->|<!DOCTYPE [\s\S]*?>|<\?[\s\S]*?\?>',
                    r'<(\w+%s)((?:%s)*?)\s*>([\s\S]*?)<\/\1>' % (
                        mistune._valid_end, mistune._valid_attr),
                    r'<\w+%s(?:%s)*?\s*\/?>' % (mistune._valid_end, mistune._valid_attr)))    

In [4]:
    error = "background-color: var(--jp-rendermime-error-background);"
    ws = "white-space: pre;"

In [5]:
    class WeaveInlineLexer(mistune.InlineLexer):
        """WeavePidginInlineLexer is responsible for updating modifying the interactive IPython state.
        """
        class grammar_class(InlineHtmlGrammar, mistune.InlineGrammar): ...

In [6]:
    class DocTestGrammar:
        doctest = doctest.DocTestParser._EXAMPLE_RE
        
    class DoctestLexer:
        def parse_doctest(self, m): self.tokens.append({'type': 'doctest', 'text': m.string[slice(*m.span())]})

In [7]:
    class GraphvizGrammar:
        graphviz = re.compile("^(di){0,1}graph\s*\{.*?\}", re.DOTALL)
    class GraphvizLexer:
        def parse_graphviz(self, m): 
            self.tokens.append({'type': 'graphviz', 'text': m.string[slice(*m.span())]})

In [8]:
    class BlockLexer(GraphvizLexer, DoctestLexer, mistune.BlockLexer): 
        class grammar_class(GraphvizGrammar, DocTestGrammar, mistune.BlockGrammar): ...
        def __init__(BlockLexer, rules=None, **kwargs): super().__init__(rules, **kwargs)
        def parse(self, text: str, rules=None) -> t.List[t.Dict]: 
            from pidgin import transform_cell
            return super().parse(transform_cell.strip_blank_lines(text))
        def parse_block_code(self, m=None): self.tokens[-1]['lang'] = super().parse_block_code(m) or ''
            
    for attr in "doctest graphviz".split():
        for key in "default_rules footnote_rules list_rules".split():[
            setattr(BlockLexer, key, list(BlockLexer.default_rules)), 
            getattr(BlockLexer, key).insert(getattr(BlockLexer, key).index('block_quote'), attr)]

In [9]:
    class Markdown(mistune.Renderer, mistune.Markdown, traitlets.HasTraits):
        run = traitlets.Bool(default_value=True)
        
        def __init__(Weave, *args, **kwargs):
            mistune.Renderer.__init__(Weave)
            mistune.Markdown.__init__(Weave, renderer=Weave, block=BlockLexer)
            Weave._codespan_buffer = []
            Weave._block_code_buffer = []

        def __enter__(Markdown): 
            Markdown.run = False
            return Markdown
        def __exit__(Markdown, *e): Markdown.run = True
            
        def codespan(Markdown, text, lang=None):
            Markdown._codespan_buffer.append(text)
            if not text.endswith('//'):
                Markdown.run_literal('\n'.join(Markdown._codespan_buffer))
                Markdown._codespan_buffer = []
            return super().codespan(text)
        
        def block_code(Markdown, text, lang=None):
            if lang is None: 
                Markdown._block_code_buffer.append(text)
                if not text.endswith('//'):
                    Markdown.run_literal('\n'.join(Markdown._block_code_buffer))
                    Markdown._block_code_buffer = []
            return super().block_code(text)
        
        def run_literal(Markdown, str):
            Markdown.log.error((22,str))
            return Markdown.run and super().run_cell(textwrap.indent(str, ' '*4), silent=True)
        
        def output_doctest(self): return self.renderer.doctest(self.token['text'])
        
        def output_graphviz(self): return self.renderer.graphviz(self.token['text'])
        
        def doctest(self, text): return super().block_code(text)
        def graphviz(self, text): 
            return __import__('graphviz').Source(text)._repr_svg_()

In [10]:
    def unload_ipython_extension(shell: IPython.core.interactiveshell.InteractiveShell):
        ...